In [ ]:
from fastai.text import * 
from fastai import *

In [ ]:
path = Path('data/wikitext-103')

Check if a line is a title of a wikipedia article or not.

In [ ]:
def istitle(line):
    return len(re.findall(r'^ = [^=]* = $', line)) != 0

Replace `<unk>` by UNK

In [ ]:
def process_unk(s):
    return UNK if s == '<unk>' else s

Read the WT103 tokens file while separating each article from the next.

In [ ]:
def read_file(filename):
    articles = []
    with open(filename, encoding='utf8') as f:
        lines = f.readlines()
    current_article = ''
    for i,line in enumerate(lines):
        current_article += line
        if i < len(lines)-2 and lines[i+1] == ' \n' and istitle(lines[i+2]):
            articles.append(current_article)
            current_article = ''
    articles.append(current_article)
    return np.array(articles)

Read the token files (download from [here](https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-v1.zip)).

In [ ]:
train = read_file(path/'wiki.train.tokens')
valid = read_file(path/'wiki.valid.tokens')
test =  read_file(path/'wiki.test.tokens')

In [ ]:
len(train), len(valid), len(test)

(28476, 60, 60)

We don't care about a separate test set, so let's join it with the training set. We put valid at the beginning because we'll use `valid_idx` to split.

In [ ]:
all_texts = np.concatenate([valid, train,test])
df = pd.DataFrame({'texts':all_texts})
df.head()

,texts
0,\n = Homarus gammarus = \n \n Homarus gammaru...
1,\n = Frank Headlam = \n \n Air Vice Marshal F...
2,\n = M @-@ 82 ( Michigan highway ) = \n \n M ...
3,\n = Shikamaru Nara = \n \n Shikamaru Nara ( ...
4,"\n = Meridian , Mississippi = \n \n Meridian ..."


In [ ]:
df.head()

,texts
0,\n = Homarus gammarus = \n \n Homarus gammaru...
1,\n = Frank Headlam = \n \n Air Vice Marshal F...
2,\n = M @-@ 82 ( Michigan highway ) = \n \n M ...
3,\n = Shikamaru Nara = \n \n Shikamaru Nara ( ...
4,"\n = Meridian , Mississippi = \n \n Meridian ..."


To release this from the RAM otherwise I can't launch the next.

In [ ]:
del train
del valid
del test

In [ ]:
data = (TextList.from_df(df, path, col='texts')
                .split_by_idx(range(0,60))
                .label_for_lm()
                .databunch())
data.save()

In [ ]:
data = TextLMDataBunch.load(path, bs=80, max_len=15)
data.show_batch()

idx,text
0,"xxbos \n = xxmaj nickel = \n \n xxmaj nickel is a chemical element with symbol xxmaj ni and atomic number 28 . xxmaj it is a silvery - white lustrous metal with a slight golden tinge . xxmaj nickel belongs to the transition metals and is hard and ductile . xxmaj pure nickel , powdered to maximize the reactive surface area , shows a significant chemical activity , but larger pieces are slow to react with air under standard conditions because an oxide layer forms"
1,"the season , xxmaj mitchell was fired and replaced by long - time assistant xxmaj jay xxmaj xxunk . xxmaj xxunk tweaked with the starting line - up to no avail as the xxmaj raptors fell to 21 – 34 prior to the xxmaj all - xxmaj star break . o ' xxmaj neal and xxmaj moon were then traded to xxmaj miami for xxmaj shawn xxmaj marion and xxmaj marcus xxmaj banks , but with the losses mounting , the xxmaj raptors soon fell"
2,") which also depicted nude figures on a boat , drew criticism of xxmaj etty for his treatment of female nudes . xxmaj the xxmaj times in early 1822 xxunk xxmaj etty , remarking that "" xxmaj we take this opportunity of advising xxmaj mr. xxmaj etty , who got some reputation for painting "" xxmaj cleopatra 's xxmaj galley "" , not to be seduced into a style which can xxunk only the most vicious taste . xxmaj naked figures , when painted with"
3,"of xxmaj johnson in the second position . xxmaj joe xxmaj nemechek was put a lap down on lap 31 ; xxmaj bobby xxmaj labonte followed xxmaj nemechek six laps later . \n xxmaj allmendinger 's lead of 1.2 seconds was reduced to nothing when the pace car was on track for the first caution . xxmaj it was given because a cover for one of a caution light 's cover had loosened . xxmaj when the drivers made pit stops , xxmaj allmendinger remained"
4,"re - shot , once with the actors saying "" philosopher 's "" and once with "" sorcerer 's "" . xxmaj the children filmed for four hours and then did three hours of schoolwork . xxmaj they developed a liking for fake facial injuries from the makeup staff . xxmaj radcliffe was initially meant to wear green contact lenses as his eyes are blue , and not green like xxmaj harry 's , but the lenses gave xxmaj radcliffe extreme irritation . xxmaj upon"
5,"using accusations of disloyalty to pressure a person to adhere to conformist politics or to discredit an opponent , xxunk civil and political rights in the name of national security , and the use of xxunk are all often referred to as mccarthyism . mccarthyism can also be synonymous with the term witch - hunt , both referring to mass hysteria and moral panic . \n \n = = mccarthyism in popular culture = = \n \n xxmaj the 1952 xxmaj arthur xxmaj miller play xxmaj the"
6,"xxmaj vena from xxup mtv and xxmaj christian xxmaj xxunk from xxmaj entertainment xxmaj weekly commented on the similarity of the music video with the xxmaj broadway musical , xxmaj rent . xxmaj vena also compared the visuals with those for xxmaj madonna 's 1986 single "" xxmaj papa xxmaj don ' t xxmaj preach "" and xxmaj michael xxmaj jackson 's 1984 single , "" xxmaj billie xxmaj jean "" . xxmaj she concluded by saying that the music video "" still manages to"
7,"séance in xxmaj xxunk in 1857 . xxmaj home wore thin shoes , easy to take off and draw on , and also cut socks that left the toes free . "" xxmaj at the appropriate moment he takes off one of his shoes and with his foot pulls a dress here , a dress there , rings a bell , knocks one way and another , and , the thing done , quickly puts his shoe on again . "" xxmaj home positioned himself"
8,", after an episode of the xxmaj american xxmaj broadcasting xxmaj company ( xxup abc ) series xxmaj grey 's xxmaj anatomy , and was the highest - rated xxup nbc series of the night . \n xxmaj myles mcnutt of xxmaj the xxup a.v. xxmaj club rated the episode with an "" a "" grade . xxmaj he felt the episode was the "" highlight "" of the season . mcnutt applauded the fact that xxmaj michael was portrayed in

In [ ]:
learn = language_model_learner(data, drop_mult=0., emb_sz=400, nh=1550, nl=4, qrnn=True, clip=0.12)
learn.fit_one_cycle(10,5e-3, moms=(0.8,0.7))

Total time: 9:43:10
epoch  train_loss  valid_loss  accuracy
1      3.602833    3.678258    0.364872  (58:25)
2      3.434083    3.567276    0.372652  (58:14)
3      3.366802    3.570619    0.370870  (58:14)
4      3.326156    3.490306    0.377881  (58:17)
5      3.281730    3.422437    0.384634  (58:24)
6      3.191068    3.323976    0.395586  (58:25)
7      3.167345    3.250199    0.403636  (58:20)
8      3.120194    3.168631    0.414734  (58:14)
9      3.085371    3.104784    0.424635  (58:25)
10     3.022416    3.087641    0.427123  (58:09)



In [ ]:
learn.save('qrnn_maj')

In [ ]:
learn = language_model_learner(data, drop_mult=0.1, clip=0.12)

In [ ]:
learn.load('lstm_maj');

In [ ]:
from fastai.callbacks.tracker import SaveModelCallback

In [ ]:
cb = SaveModelCallback(learn)

In [ ]:
learn.fit_one_cycle(5,1e-3, moms=(0.8,0.7), callbacks=[cb], pct_start=0.1)

Total time: 4:50:51
epoch  train_loss  valid_loss  accuracy
1      3.132123    3.166337    0.415251  (58:10)
2      3.090379    3.134404    0.419966  (58:14)
3      3.021402    3.108257    0.423943  (58:11)
4      3.008096    3.088369    0.427395  (58:20)
5      2.961441    3.072581    0.429707  (57:54)



In [ ]:
learn.save('qrnn_maj1')

In [ ]:
learn.validate(learn.data.valid_dl)

[3.062496, tensor(0.4311)]